# AI CopyWriter

Прототип перефразирования и/или распространения текста на основе ruGPT3Large.

Ссылка на COLAB https://colab.research.google.com/github/dilyararimovna/text_expansion/blob/main/Text_Expansion.ipynb

### Установим необходимые зависимости:

In [1]:
!pip3 install urllib3==1.25.4

!pip3 install transformers==2.8.0

!pip install pymorphy2==0.8

!pip install spacy==2.1.9

!python3 -m spacy download xx_ent_wiki_sm

!pip install deeppavlov==0.13.0

!pip3 uninstall tensorflow --yes

!pip3 install tensorflow-gpu==1.15.2

!python3 -m  deeppavlov install syntax_ru_syntagrus_bert

! pip3 install jsonlines

     |████████████████████████████████| 125 kB 5.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 563 kB 5.8 MB/s 
     |████████████████████████████████| 5.6 MB 30.7 MB/s 
     |████████████████████████████████| 1.2 MB 3.3 MB/s 
     |████████████████████████████████| 895 kB 36.6 MB/s 
     |████████████████████████████████| 132 kB 31.1 MB/s 
     |████████████████████████████████| 8.7 MB 36.0 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 46 kB 2.0 MB/s 
     |████████████████████████████████| 7.1 MB 8.4 MB/s 
  

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0
     |████████████████████████████████| 410.9 MB 35 kB/s 
     |████████████████████████████████| 3.8 MB 35.1 MB/s 
     |████████████████████████████████| 503 kB 46.7 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=3352356792409563636cbebdf3aa3b7ccbce5fdf235bb2f00ce3cb7943dd75a3
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard

### Now several cells with the code.

In [1]:
import jsonlines
import json
import os
import random
import re
import string  
from copy import deepcopy
from pathlib import Path

import numpy as np
import pandas as pd
import pymorphy2
import spacy
import xx_ent_wiki_sm
from deeppavlov import build_model, configs

morph = pymorphy2.MorphAnalyzer()

nlp = xx_ent_wiki_sm.load()
nlp.add_pipe(nlp.create_pipe('sentencizer'), first=True)

syntax_model = build_model(configs.syntax.syntax_ru_syntagrus_bert, download=True)


/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
2022-05-01 08:20:20.542 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/morpho_tagger/UD2.3/ru_syntagrus.tar.gz download because of matching hashes
2022-05-01 08:20:23.331 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2022-05-01 08:20:25.608 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/syntax_parser/syntax_ru_syntagrus_bert.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloa

2022-05-01 08:20:27.986 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/syntax_ru_syntagrus/deps.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please 

2022-05-01 08:20:52.658 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/syntax_ru_syntagrus/model_joint]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/syntax_ru_syntagrus/model_joint


In [2]:
from transformers import GPT2LMHeadModel,GPT2Tokenizer
import torch

tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
model.to(device)
print("Success!")

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,



Success!


In [4]:
russian_restricted_pronouns = "я мной меня мною мне мы нас нам нами ты тебя тебе тобою тобой вы вас вам вами".split()
extra_marks = re.compile(r"&[a-zA-Z0-9;]+")
expanding_startings = [
    "В то же время", 
    "Это так", 
    "Действительно,", 
    "Потому что", 
    "А главное,", 
    "При этом",
    "В связи с этим",
    "Кроме того,",
    "Интересно, что",
    "Также",
    ]


def join_words_in_or_pattern(words):
    return "(" + "|".join([r'\b%s\b' % word for word in words]) + ")"


RU_PRONOUNS = re.compile(join_words_in_or_pattern(russian_restricted_pronouns), re.IGNORECASE)


def generate_rugpt3large(prompt_text, return_only_predicted=False,
                         till_new_string=True, generate_only_one_sent=True, 
                         length=100, temperature=1.0, k=10, p=0.9, repetition_penalty=1.0,
                         num_return_sequences=1, stop_token="</s>"):
    global device
    total_sequence = ""
    if num_return_sequences > 1:
        all_sequences = []
    
    encoded_prompt = tokenizer.encode(
        prompt_text, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(device)

    output_sequences = model.generate(
                input_ids=encoded_prompt,
                max_length=length + len(encoded_prompt[0]),
                temperature=temperature,
                top_k=k,
                top_p=p,
                repetition_penalty=repetition_penalty,
                do_sample=True,
                num_return_sequences=num_return_sequences,
            )
    
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        generated_sequence = generated_sequence.tolist()
        
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        text = text[: text.find(stop_token) if stop_token else None]

        total_sequence = (text[len(tokenizer.decode(
            encoded_prompt[0], clean_up_tokenization_spaces=True)) :])
        total_sequence = re.sub(r"[\"'«»]", "", total_sequence)
        total_sequence = total_sequence.strip()
        
        if till_new_string:
            total_sequence = total_sequence[:total_sequence.find("\n")].strip()
            
        if generate_only_one_sent:
            total_sequence = " ".join([sent.text for sent in nlp(total_sequence).sents][:1])
            
        if len(total_sequence) > 1 and prompt_text[-1] in [".", "!", "?", "…"]:
            total_sequence = total_sequence[0].upper() + total_sequence[1:]
            
        if not return_only_predicted:
            if len(total_sequence) > 1 and total_sequence[0] in [".", "!", "?", "…", ","]:
                total_sequence = prompt_text + total_sequence
            else:
                total_sequence = prompt_text + " " + total_sequence
        else:
            if len(total_sequence) > 1 and total_sequence[0] in [".", "!", "?", "…", ","]:
                total_sequence = total_sequence
            else:
                total_sequence = " " + total_sequence
        
        if num_return_sequences > 1 and len(total_sequence) > 0:
            all_sequences.append(total_sequence)
            
    if num_return_sequences > 1:
        return all_sequences
    else:
        return total_sequence
    

def get_nsubjects(text):
    nsubjects = []
    for parse in syntax_model([text]):
        for row in parse.split("\n"):
            if "nsubj" in row:
                nsubjects.append(row.split("\t")[1])

    return nsubjects


def is_satisfying(sent):
    doc = nlp(sent)
    if len(doc.ents) > 0:
        return False
    
    ntokens = len(sent.split())
    if 15 <= ntokens or ntokens < 5:
        return False
    if re.search(RU_PRONOUNS, sent):
        return False
    
    return True


def generate_paraphrase(text):
    predictions = generate_rugpt3large(f"{text} Перефразирую:", 
                                       return_only_predicted=True, 
                                       num_return_sequences=10)
    predictions = [p.strip() for p in predictions]
    predictions = [p for p in predictions if len(p) > 1]
    original_tokens = set(text.split())
    nsubjects = set(get_nsubjects(text))
    
    total_n_tokens = []
    total_same_tokens = []
    total_same_nsubjects = []
    
    for pred_par in predictions:
        pred_tokens = pred_par.split()
        
        len_pred_tokens = len(pred_tokens)
        len_same_tokens = len(original_tokens.intersection(set(pred_tokens)))
        len_same_nsubjs = len(nsubjects.intersection(set(pred_tokens)))
        
        total_n_tokens.append(len_pred_tokens)
        total_same_tokens.append(len_same_tokens)
        total_same_nsubjects.append(len_same_nsubjs)
    
    targets = [st * sn / tt for st, sn, tt in 
               zip(total_same_tokens, total_same_nsubjects, total_n_tokens)]
    # if all([el <= 1 for el in total_same_nsubjects]):
    if sum(total_same_nsubjects) == 0:
        targets = [st / tt for st, sn, tt in 
                   zip(total_same_tokens, total_same_nsubjects, total_n_tokens)]
    targets = [t if predictions[i] != text else 0 for i, t in enumerate(targets)]
    best_pred_id = np.argmax(targets)
    
    new_sent_text = predictions[best_pred_id]
    if len(new_sent_text) > 1:
        if new_sent_text[-1] not in [".", "!", "?", "…"]:
            new_sent_text = new_sent_text + "."
        new_sent_text = new_sent_text[0].upper() + new_sent_text[1:]
    else:
        new_sent_text = text
    return new_sent_text

    
def expand_text(text, paraphrase=False, max_history_sents=5):
    expanded = deepcopy(text)
    
    doc = nlp(text)
    expanded_sents = []
    sents_texts = [sent.text for sent in doc.sents if len(sent.text) > 1]
    for sent_id, sent in enumerate(doc.sents):
        if len(sent.text) <= 1:
            continue
        expanded_sents.append(sent.text)
        
        expand_sent = ""
        if sent_id < len(sents_texts) - 1 and {"CONJ"} in morph.parse(sents_texts[sent_id + 1].split()[0])[0].tag:
            pass
        else:
            expanding_start = random.choice(expanding_startings)
            context = " ".join(expanded_sents[-max_history_sents:])
            generated = generate_rugpt3large(f"{context} {expanding_start}",
                                             return_only_predicted=True, num_return_sequences=5)
            
            for gen in generated:
                if is_satisfying(gen):
                    expand_sent = f"{expanding_start}{gen}".strip()
                    if expand_sent[-1] not in [".", "!", "?", "…"]:
                        expand_sent = expand_sent + "."
                    break
        
        new_subtext = deepcopy(sent.text)
        if paraphrase:
            new_sent_text = generate_paraphrase(sent.text).strip()
            if len(new_sent_text) > 1:
                if new_sent_text[-1] not in [".", "!", "?", "…"]:
                    new_sent_text = new_sent_text + "."
                new_sent_text = new_sent_text[0].upper() + new_sent_text[1:]
            else:
                new_sent_text = deepcopy(sent.text)
                
            if len(expand_sent) > 0:
                new_subtext = f"{new_sent_text} {expand_sent}"
                expanded_sents.append(f"{expand_sent}")
            else:
                new_subtext = f"{new_sent_text}"
        else:
            if len(expand_sent) > 0:
                new_subtext = f"{sent.text} {expand_sent}"
                expanded_sents.append(f"{expand_sent}")
                
        expanded = expanded.replace(sent.text, new_subtext)
        
                
    expanded = expanded.strip()
    if abs(len(expanded) - len(text)) < 10:
        expanded = expand_text(text, max_history_sents=5)
    return expanded

In [5]:
def paraphrase_and_expand_text(text, paraphrase=False, expand=False, 
                               max_history_sents=5):
    text = text.strip()
    if expand:
        paraphrased_expanded = expand_text(text, 
                                           paraphrase=paraphrase, 
                                           max_history_sents=max_history_sents)
    elif paraphrase:
        paraphrased_expanded = ""
        doc = nlp(text)
        for sent_id, sent in enumerate(doc.sents):
            if len(sent.text) <= 1:
                continue

            if paraphrase:
                par_sent = generate_paraphrase(sent.text)
                paraphrased_expanded += f" {par_sent}"
    paraphrased_expanded = re.sub(extra_marks, "", paraphrased_expanded)
    return paraphrased_expanded.strip()

# DEMO Usage

Итак, все зависимости установлены, функции определены. Самое время просто воспользоваться DEMO. 
Пользователь демонстрации может изменять список текстов, и главное параметры `paraphrase` и `expand`. Параметр `paraphrase` отвечает за то, будет ли исходный текст перефразирован, а параметр `expand` показывает следует ли модели дополнить текст. Эти режимы могут быть включены как по отдельности, так и вместе.

In [6]:
for text in [
             """На мой взгляд, лето – прекраснейшее время года, поскольку очень тепло, вокруг все красиво, и у нас имеется отличная возможность отдохнуть. 
             А еще лето – это время каникул и отпусков. 
             Мы можем отправиться куда угодно, замечательно провести время у водоема, в тени изумрудной зелени, плескаться в теплой воде. 
             А можно отправиться в горы, носиться на велосипеде или играть во всевозможные игры.""",
             """Лето – это всегда особенное время года и каждый человек старается сделать его запоминающимся. 
             Тут пригодится умение замечать простые мелочи, впитывать в себя солнечные и радостные моменты каждого дня. 
             Даже если на улице идет дождь, поездка на море отменяется, а любимое мороженное в магазине закончилось. 
             Ведь это всего лишь отдельные дни и в целом лето это калейдоскоп ярких переживаний, который мы держим в своих руках.""",
             """Тысячи капель прохладной воды падают на лицо и стекают по волосам и телу, солнце нежно щекотит кожу, а ты вдыхаешь запах сочной листвы. 
             И вы не просто мажете тело обёртыванием или наносите на лицо сыворотку, вы совершаете ритуал любви к себе, своей коже и своему телу. 
             Это волшебное время, когда девушка находиться наедине с собой. Она вся в своих мыслях, она приводит не только себя в порядок, но и свой разум. 
             Восстанавливается после тяжелого дня или наоборот, только готовится к нему, чтобы быть бодрой и энергичной."""]:

    rewritten_text = paraphrase_and_expand_text(text, paraphrase=True, expand=True)
    print(f"Original Text: {text}\n\nRewritten Text: {rewritten_text}\n\n\n")

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Original Text: На мой взгляд, лето – прекраснейшее время года, поскольку очень тепло, вокруг все красиво, и у нас имеется отличная возможность отдохнуть. 
             А еще лето – это время каникул и отпусков. 
             Мы можем отправиться куда угодно, замечательно провести время у водоема, в тени изумрудной зелени, плескаться в теплой воде. 
             А можно отправиться в горы, носиться на велосипеде или играть во всевозможные игры.

Rewritten Text: На мой взгляд, лето – прекраснейшее время года, поскольку очень тепло, вокруг все красиво, и у нас имеется отличная возможность отдохнуть. 
             А еще лето – это время каникул и отпусков. При этом можно не только отдохнуть, но и набраться сил, здоровья и энергии. 
             Мы можем отправиться куда угодно, замечательно провести время у водоема, в тени изумрудной зелени, плескаться в теплой воде. 
             А можно отправиться в горы, носиться на велосипеде или играть во всевозможные игры. А главное, что в этом врем

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Original Text: Лето – это всегда особенное время года и каждый человек старается сделать его запоминающимся. 
             Тут пригодится умение замечать простые мелочи, впитывать в себя солнечные и радостные моменты каждого дня. 
             Даже если на улице идет дождь, поездка на море отменяется, а любимое мороженное в магазине закончилось. 
             Ведь это всего лишь отдельные дни и в целом лето это калейдоскоп ярких переживаний, который мы держим в своих руках.

Rewritten Text: Лето – это время, когда можно не только наслаждаться теплом солнца, но и заниматься спортом. Кроме того, каждый человек желает, чтобы лето запомнилось на долгие годы. Счастье не в деньгах. А главное, не стоит забывать о том, что лето – это пора отпусков. Если на улице идет дождь, то поездка на море отменяется, а любимое мороженное в магазине заканчивается. Лето - это всего лишь отдельные дни и в целом лето - это калейдоскоп ярких впечатлений. А главное, лето – это время, когда можно позволить себе в

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Original Text: Тысячи капель прохладной воды падают на лицо и стекают по волосам и телу, солнце нежно щекотит кожу, а ты вдыхаешь запах сочной листвы. 
             И вы не просто мажете тело обёртыванием или наносите на лицо сыворотку, вы совершаете ритуал любви к себе, своей коже и своему телу. 
             Это волшебное время, когда девушка находиться наедине с собой. Она вся в своих мыслях, она приводит не только себя в порядок, но и свой разум. 
             Восстанавливается после тяжелого дня или наоборот, только готовится к нему, чтобы быть бодрой и энергичной.

Rewritten Text: – Тысячи капель прохладной воды падают на лицо и стекают по волосам и телу, солнце нежно щекотит кожу, а ты вдыхаешь запах сочной листвы. Вы не просто мажете тело обёртыванием или наносите на лицо сыворотку, вы совершаете ритуал любви к себе, своей коже и своему телу. Потому что любовь – это не просто желание нравиться другим, это желание отдавать, дарить, заботиться. Девушка наедине с собой. Также, как